In [1]:
import pandas as pd

In [2]:
# 데이터 경로
train_data = pd.read_csv('./data/train.csv', index_col='id')
test_data = pd.read_csv('./data/test.csv', index_col='id')

# 01. 데이터 합치기

In [3]:
all_data = pd.concat([train_data, test_data])  # Train_data와 Test_data 합치기

In [4]:
all_data.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0.0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0.0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0.0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1.0
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0.0


In [5]:
new_data = all_data.drop('target', axis=1)
new_data.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8


In [6]:
from sklearn.preprocessing import OneHotEncoder

In [7]:
encoder = OneHotEncoder() # 원-핫 인코딩 객체 생성

In [8]:
all_data_encorder = encoder.fit_transform(new_data)  # 전체 데이터에 대하여 원-핫 인코딩 적용

# 02. 데이터 나누기 ( train, test )

In [9]:
num_train = len(train_data)
num_train

300000

In [10]:
all_data_encorder.shape

(500000, 16552)

In [11]:
X_train = all_data_encorder[:num_train]
X_test = all_data_encorder[num_train:]

In [12]:
X_train.shape

(300000, 16552)

In [13]:
X_test.shape

(200000, 16552)

In [14]:
Y_train = train_data['target']
Y_train.shape

(300000,)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train_new, X_test_new, Y_train_new, Y_test_new = train_test_split(X_train, Y_train, test_size=0.1, stratify=Y_train, random_state=10)

In [17]:
print(X_train_new.shape)
print(Y_train_new.shape)
print(X_test_new.shape)
print(Y_test_new.shape)

(270000, 16552)
(270000,)
(30000, 16552)
(30000,)


# 03. 모델 훈련

In [18]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression(max_iter=1000, random_state=42)

In [19]:
logistic_model.fit(X_train_new, Y_train_new) # 모델 훈련

LogisticRegression(max_iter=1000, random_state=42)

# 04. 모델 성능 검증

In [20]:
logistic_model.predict_proba(X_test)

array([[0.69173111, 0.30826889],
       [0.30069014, 0.69930986],
       [0.93208654, 0.06791346],
       ...,
       [0.59378427, 0.40621573],
       [0.50791524, 0.49208476],
       [0.81365144, 0.18634856]])

In [21]:
logistic_model.predict(X_test_new)

array([1, 0, 0, ..., 1, 1, 0], dtype=int64)

In [22]:
# 검증 데이터를 활용한 타깃 예측
y_test_preds =  logistic_model.predict_proba(X_test_new)[:, 1]

In [23]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

# 검증 데이터 ROC AUC
roc_auc = roc_auc_score(Y_test_new, y_test_preds)

print(f'검증 데이터 ROC AUC : {roc_auc:.4f}')

검증 데이터 ROC AUC : 0.7965
